In [ ]:
import pandas as pd
import pymongo
import json
import csv

In [ ]:
def leggi_dati_rubrica():

    file_prof = "data/tabelle/RubricaMIURStatEnriched.csv"

    prof_miur = pd.read_csv(file_prof, sep=',')

    prof_data = prof_miur[['id', 'Cognome e Nome', 'ror', 'id_oa']]

    print(prof_data.head(5))

In [9]:
leggi_dati_rubrica()

   id          Cognome e Nome                        ror  \
0   0     ABASTANTE Francesca                        NaN   
1   1       ABATANGELO Chiara                        NaN   
2   2        ABATANGELO Laura  https://ror.org/01nffqt88   
3   3       ABATANGELO Nicola                        NaN   
4   4  ABATE Andrea Francesco                        NaN   

                            id_oa  
0                             NaN  
1                             NaN  
2  https://openalex.org/I93860229  
3                             NaN  
4                             NaN  


In [6]:
def leggi_authors_oa():

    file_out = "data/tabelle/authors_ridotto_test.json"

    fields_to_extract = ['id', 'orcid', 'display_name', 'display_name_alternatives', 'affiliations']

    # Lista per memorizzare i dati filtrati
    filtered_data = []

    # Apri il file JSON Lines
    with open('data/raw_data/openalex/oa_authors_test.jsonl', 'r', encoding='utf-8') as jsonl_file:
        # Leggi il file JSONL riga per riga
        for line in jsonl_file:
            try:
                # Decodifica ogni riga come un oggetto JSON
                data = json.loads(line.strip())

                # Estrai solo i campi desiderati
                filtered_entry = {key: data[key] for key in fields_to_extract if key in data}
                
                # Aggiungi i dati filtrati alla lista
                filtered_data.append(filtered_entry)
            except json.JSONDecodeError as e:
                print(f"Errore nella decodifica della riga: {line} - {e}")

    # Salva i dati filtrati in un nuovo file JSON
    with open(file_out, 'w', encoding='utf-8') as json_out_file:
        json.dump(filtered_data, json_out_file, ensure_ascii=False, indent=4)




In [7]:
leggi_authors_oa()

In [ ]:
def matching_professori():
    """
Leggi id 
"""
    uni_tobe_matched = 

In [ ]:
def get_authors_by_institution_id(institution_id, collection):
    pipeline = [
        {"$unwind": "$affiliations"},
        {"$match": {"affiliations.institution.id": institution_id}},
        {"$project": {
            "id": 1,
            "orcid": 1,
            "display_name": 1,
            "display_name_alternatives": 1
        }}
    ]
    
    try:
        cursor = collection.aggregate(pipeline)
        authors_dict = {}
        
        for doc in cursor:
            author_id = doc.get("id")
            if not author_id:
                continue
                
            alternatives = doc.get("display_name_alternatives", [])
            if not isinstance(alternatives, list):
                alternatives = []
            
            authors_dict[author_id] = {
                "id": author_id,
                "orcid": doc.get("orcid"),
                "display_name": doc.get("display_name", ""),
                "display_name_alternatives": alternatives
            }
        
        print(f"Found {len(authors_dict)} authors for institution {institution_id}")
        return authors_dict
        
    except Exception as e:
        print(f"Error: {e}")
        return {}

In [ ]:
from pymongo import MongoClient
import json

# Connessione a MongoDB
def setup_mongo_connection():
    try:
        client = MongoClient('mongodb://localhost:27017/')  # Modifica con i tuoi parametri
        db = client.unisurf  
        collection = db.oa_authors_test  
        
        # Test connessione
        client.admin.command('ping')
        print("MongoDB connection successful!")
        
        return collection
    except Exception as e:
        print(f"MongoDB connection failed: {e}")
        return None

setup_mongo_connection()

MongoDB connection successful!


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'unisurf'), 'oa_authors_test')

In [22]:
def get_mongo_collection():
    client = MongoClient('mongodb://localhost:27017/')  # I tuoi parametri
    db = client.unisurf  # Nome del tuo database
    collection = db.oa_authors_test  # Nome della collezione
    return collection

In [23]:
get_authors_by_institution_id('https://openalex.org/I189158943', get_mongo_collection())

Found 4894 authors for institution https://openalex.org/I189158943


{'https://openalex.org/A5075525738': {'id': 'https://openalex.org/A5075525738',
  'orcid': 'https://orcid.org/0000-0003-1441-897X',
  'display_name': 'Carlo La Vecchia',
  'display_name_alternatives': ['Carlo La Vecchi',
   'CARLO LA VACCHIA',
   'CarloLa Vecchia',
   'P. G. Crosignani',
   'Carlo Vecchia',
   'C. Vecchia',
   'Carolo La Vecchia',
   'Carlo La‐Vecchia',
   'C. L. Vecchia',
   'Carlo La Vecchia',
   'Crrlo La Vecchia',
   'Carlo L. Vecchia',
   'Carla La Vecchia',
   'La Vecchia C',
   'Caxlo La Vecchia',
   'Carlo Lavecchia',
   'Carlo La Veccbia',
   'Carlo La Vecchila',
   'C. Lavecchia',
   'C.V.B. La Vecchia',
   'Carlo La Vecchir',
   'C. La Vecchia',
   'Claudia La Vecchia']},
 'https://openalex.org/A5108167190': {'id': 'https://openalex.org/A5108167190',
  'orcid': 'https://orcid.org/0000-0002-0227-1301',
  'display_name': 'P. Govoni',
  'display_name_alternatives': ['Pietro Govoni', 'P. Govoni']},
 'https://openalex.org/A5106314221': {'id': 'https://openalex.or

In [ ]:
collection = 
# Test 1: Conta quanti documenti hanno affiliations
count_with_affiliations = collection.count_documents({"affiliations": {"$exists": True}})
print(f"Documents with affiliations: {count_with_affiliations}")

# Test 2: Prova la query senza unwind
test_query = {"affiliations.institution.id": "https://openalex.org/I189158943"}
count_direct = collection.count_documents(test_query)
print(f"Direct query count: {count_direct}")

# Test 3: Trova institution IDs disponibili
pipeline = [
    {"$unwind": "$affiliations"},
    {"$group": {"_id": "$affiliations.institution.id"}},
    {"$limit": 5}
]
available_ids = list(collection.aggregate(pipeline))
print(f"Available institution IDs: {[x['_id'] for x in available_ids]}")

In [ ]:
def load_professor_stack_forid(file_path, id_ateneo):
   """
   Carica il CSV dei professori e restituisce una lista (stack)
   
   Args:
       file_path (str): Percorso del file CSV
   
   Returns:
       list: Lista di dizionari con i dati dei professori
   """
   try:
       df = pd.read_csv(file_path)
       
       # Converte in lista di dizionari
       professor_stack = []
       for _, row in df.iterrows():
           if row["id_oa"] == id_ateneo:
            professor_stack.append({
                "id": row["id"],
                "fascia": row["Fascia"],
                "cognome": row["Cognome e Nome"],  # O row["Cognome"] se separato
                "nome_completo": row["Cognome e Nome"],
                "ateneo": row["Ateneo"],
                "id_oa_ateneo": row["id_oa"]
           })
       
       print(f"Loaded {len(professor_stack)} professors")
       return professor_stack
       
   except Exception as e:
       print(f"Error loading CSV: {e}")
       return []

# Uso:
prof_stack = load_professor_stack_forid("data/tabelle/RubricaMIURStatEnriched.csv")

Loaded 61188 professors


In [27]:
def get_university_list():
   df = pd.read_csv("data/tabelle/RubricaMIURStatEnriched.csv")
   return df['id_oa'].dropna().unique().tolist()

In [ ]:
def main_professor_matcher():
    atenei_id = get_university_list()

    for ateneo_id in atenei_id:
        load_professor_stack_forid("data/tabelle/RubricaMIURStatEnriched.csv", ateneo_id)

['https://openalex.org/I93860229', 'https://openalex.org/I66752286']


In [ ]:
def analyze_names_in_dictionary(authors_dict):
    """
    Analizza i nomi nel dizionario degli autori per identificare pattern,
    lunghezza dei token, caratteri speciali, alfabeti non latini, etc.
    """
    
    analysis = {
        'total_authors': len(authors_dict),
        'display_names': {'tokens': [], 'chars': [], 'scripts': []},
        'alternatives': {'tokens': [], 'chars': [], 'scripts': []},
        'strange_chars': [],
        'non_latin_names': [],
        'empty_names': 0,
        'very_long_names': [],
        'stats': {}
    }
    
    # Pattern per identificare caratteri non-ASCII
    non_ascii_pattern = re.compile(r'[^\x00-\x7F]')
    
    for author_id, author_data in authors_dict.items():
        
        # Analizza display_name
        display_name = author_data.get('display_name', '')
        if display_name:
            tokens = display_name.split()
            analysis['display_names']['tokens'].append(len(tokens))
            analysis['display_names']['chars'].append(len(display_name))
            
            # Identifica script/alfabeto
            script = identify_script(display_name)
            analysis['display_names']['scripts'].append(script)
            
            # Caratteri strani
            if non_ascii_pattern.search(display_name):
                analysis['strange_chars'].append({
                    'author_id': author_id,
                    'name': display_name,
                    'type': 'display_name'
                })
            
            # Nomi non latini
            if script != 'Latin':
                analysis['non_latin_names'].append({
                    'author_id': author_id,
                    'name': display_name,
                    'script': script,
                    'type': 'display_name'
                })
            
            # Nomi molto lunghi (>50 caratteri)
            if len(display_name) > 50:
                analysis['very_long_names'].append({
                    'author_id': author_id,
                    'name': display_name,
                    'length': len(display_name),
                    'type': 'display_name'
                })
        else:
            analysis['empty_names'] += 1
        
        # Analizza display_name_alternatives
        alternatives = author_data.get('display_name_alternatives', [])
        for alt_name in alternatives:
            if alt_name:
                tokens = alt_name.split()
                analysis['alternatives']['tokens'].append(len(tokens))
                analysis['alternatives']['chars'].append(len(alt_name))
                
                script = identify_script(alt_name)
                analysis['alternatives']['scripts'].append(script)
                
                if non_ascii_pattern.search(alt_name):
                    analysis['strange_chars'].append({
                        'author_id': author_id,
                        'name': alt_name,
                        'type': 'alternative'
                    })
                
                if script != 'Latin':
                    analysis['non_latin_names'].append({
                        'author_id': author_id,
                        'name': alt_name,
                        'script': script,
                        'type': 'alternative'
                    })
                
                if len(alt_name) > 50:
                    analysis['very_long_names'].append({
                        'author_id': author_id,
                        'name': alt_name,
                        'length': len(alt_name),
                        'type': 'alternative'
                    })
    
    # Calcola statistiche
    analysis['stats'] = calculate_stats(analysis)
    
    return analysis

def identify_script(text):
    """Identifica il sistema di scrittura predominante nel testo"""
    if not text:
        return 'Unknown'
    
    script_counts = Counter()
    
    for char in text:
        if char.isalpha():
            try:
                script = unicodedata.name(char).split()[0]
                script_counts[script] += 1
            except ValueError:
                script_counts['Unknown'] += 1
    
    if not script_counts:
        return 'No_Letters'
    
    most_common_script = script_counts.most_common(1)[0][0]
    
    # Raggruppa script simili
    if most_common_script in ['LATIN', 'LATIN-1']:
        return 'Latin'
    elif most_common_script in ['CYRILLIC']:
        return 'Cyrillic'
    elif most_common_script in ['GREEK']:
        return 'Greek'
    elif most_common_script in ['ARABIC']:
        return 'Arabic'
    elif most_common_script in ['CJK']:
        return 'Chinese/Japanese/Korean'
    else:
        return most_common_script

def calculate_stats(analysis):
    """Calcola statistiche riassuntive"""
    stats = {}
    
    # Statistiche sui token
    if analysis['display_names']['tokens']:
        stats['display_name_tokens'] = {
            'min': min(analysis['display_names']['tokens']),
            'max': max(analysis['display_names']['tokens']),
            'avg': sum(analysis['display_names']['tokens']) / len(analysis['display_names']['tokens']),
            'distribution': Counter(analysis['display_names']['tokens'])
        }
    
    if analysis['alternatives']['tokens']:
        stats['alternatives_tokens'] = {
            'min': min(analysis['alternatives']['tokens']),
            'max': max(analysis['alternatives']['tokens']),
            'avg': sum(analysis['alternatives']['tokens']) / len(analysis['alternatives']['tokens']),
            'distribution': Counter(analysis['alternatives']['tokens'])
        }
    
    # Statistiche sui caratteri
    if analysis['display_names']['chars']:
        stats['display_name_chars'] = {
            'min': min(analysis['display_names']['chars']),
            'max': max(analysis['display_names']['chars']),
            'avg': sum(analysis['display_names']['chars']) / len(analysis['display_names']['chars'])
        }
    
    # Distribuzione degli script
    stats['script_distribution'] = {
        'display_names': Counter(analysis['display_names']['scripts']),
        'alternatives': Counter(analysis['alternatives']['scripts'])
    }
    
    return stats

def print_analysis_report(analysis):
    """Stampa un report leggibile dell'analisi"""
    
    print("="*60)
    print("ANALISI NOMI AUTORI")
    print("="*60)
    
    print(f"\n📊 STATISTICHE GENERALI:")
    print(f"   Totale autori: {analysis['total_authors']}")
    print(f"   Nomi vuoti: {analysis['empty_names']}")
    print(f"   Nomi con caratteri non-ASCII: {len(analysis['strange_chars'])}")
    print(f"   Nomi non-latini: {len(analysis['non_latin_names'])}")
    print(f"   Nomi molto lunghi (>50 char): {len(analysis['very_long_names'])}")
    
    stats = analysis['stats']
    
    # Statistiche token display_names
    if 'display_name_tokens' in stats:
        token_stats = stats['display_name_tokens']
        print(f"\n🏷️  TOKEN DISPLAY_NAMES:")
        print(f"   Min token: {token_stats['min']}")
        print(f"   Max token: {token_stats['max']}")
        print(f"   Media token: {token_stats['avg']:.1f}")
        print(f"   Distribuzione più comune:")
        for tokens, count in token_stats['distribution'].most_common(5):
            print(f"      {tokens} token: {count} nomi")
    
    # Distribuzione script
    print(f"\n🌍 DISTRIBUZIONE ALFABETI:")
    print("   Display names:")
    for script, count in stats['script_distribution']['display_names'].most_common():
        print(f"      {script}: {count}")
    
    # Esempi nomi strani
    if analysis['non_latin_names']:
        print(f"\n🔤 ESEMPI NOMI NON-LATINI:")
        for i, item in enumerate(analysis['non_latin_names'][:5]):
            print(f"   {i+1}. {item['name']} (Script: {item['script']})")
    
    # Esempi nomi lunghi
    if analysis['very_long_names']:
        print(f"\n📏 ESEMPI NOMI MOLTO LUNGHI:")
        for i, item in enumerate(analysis['very_long_names'][:5]):
            print(f"   {i+1}. {item['name']} ({item['length']} caratteri)")